In [1]:
import pandas as pd

GYM = '/kaggle/input/gym-members-exercise-dataset/gym_members_exercise_tracking.csv'

TARGET = 'Workout_Type'

df = pd.read_csv(filepath_or_buffer=GYM)
df = pd.get_dummies(data=df, columns=['Gender',])
df.head()

,Age,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,Gender_Female,Gender_Male
0,56,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20,False,True
1,46,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00,True,False
2,32,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71,True,False
3,25,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41,False,True
4,38,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39,False,True


In [2]:
df[TARGET].value_counts().tolist()

[258, 255, 239, 221]